In [10]:
import sagemaker
import boto3
from sagemaker import get_execution_role

import pandas as pd
import numpy as np
import sys

sys.path.extend(['../src', '../config'])

region = boto3.Session().region_name
session = sagemaker.Session()
sm = boto3.Session().client(service_name='sagemaker',region_name=region)

import helpers.instance as ins
import helpers.s3 as s3_helper
import helpers.utils as ut
import helpers.athena as at

# READ YAML FILES AND STORE RELEVANT PATHS in dictionaries
dataset_cfg = ins.read_config('../config/datasets.yaml') 
config_cfg = ins.read_config('../config/config.yaml')
sql_cfg = ins.read_config('../config/sql.yaml')

In [11]:
dataset_cfg['raw']['base_query']['input_paths']

's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Athena_query/base_query.txt'

In [12]:
# passing Athena variables
database=['ptledw_playarea', 'ptl_maintenance', 'ptl_connfleet', 'proactive_maint_db', 'ptl_adhoc']
workgroup='WG-Advanalytics'
tablename=['d_location_master','constellation_rules','ufch_depup', 'archive_pd_inference_scenario_output', 'corp_foilanl']
#query_string = 'select * from ptl_maintenance.constellation_rules'
#query_string = 'select * from {}.{}'.format(database[0],tablename[0])
# 'select * from ptledw_playarea.d_location_master'

In [13]:
#Query string from a file, essentially
#region = boto3.Session().region_name
s3 = boto3.client('s3', region)
bucket_name = dataset_cfg['bucket']
#file = "Projects/Unit_Sale_Risk_Interns/Data/Athena_query/base_query.txt"
file = dataset_cfg['raw']['base_query']['input_file']
file_obj = s3.get_object(Bucket=bucket_name,Key=file) #retrieves from S3
file_data = file_obj['Body'].read() #reads contents of file
query_string = file_data.decode() #method used to decode strings, which the file is

In [17]:
query_string = open('../sql/base_query.sql').read()
query_string = query_string.replace('\n',' ')
query_string = query_string.replace('\t',' ')
query_string
#query_string = 'select * from ptl_maintenance.constellation_rules'

"SELECT  unit.UNIT_NUM,  agg.PARTITION_KEY AS PARTITION_KEY,  veh.VEHICLE_AGE_TOTAL_MONTHS,  loc.REGN_NUM || '-' || loc.REGN_NM AS REGION,  loc.AREA_NUM || '-' || loc.AREA_NM AS AREA,  loc.RDIST_NUM || '-' || loc.RDIST_SUFX || '-' || loc.RDIST_NM AS DISTRICT,  loc.DIST_NUM || '-' || loc.DIST_SUFX || '-' || loc.DIST_NM AS LOCATION,  unit.UNIT_MAKE_CODE,  unit.UNIT_MODEL,  unit.UNIT_MODEL_YEAR,  unit.UNIT_CATEGORY,  unit.UNIT_SIZE,  unit.UNIT_BODY_TYPE,  unit.UNIT_FUEL_TYPE,  unit.ACT_VEH_INSERVDT,  acc.ACCOUNT_CODE_GROUP1 AS PRODUCT_LINE,  unit.UNIT_SOLD_DATE ,  'model_build' AS flag_datatype,  MAX(agg.LTD_MILES) AS MILES FROM  ptledw_playarea.F_AGG_UNITCOST_EOM agg  JOIN ptledw_playarea.D_UNIT unit ON   unit.UNIT_KEY = agg.UNIT_KEY  JOIN ptledw_playarea.D_VEHICLE_AGE veh ON   agg.ACT_VEHICLE_AGE_KEY = veh.vehicle_age_key  JOIN ptledw_playarea.D_ACCOUNT_CODE acc ON   agg.ACCOUNT_CODE_KEY = acc.ACCOUNT_CODE_KEY  JOIN ptledw_playarea.D_LOCATION_MASTER loc ON   agg.LOCATION_MASTER_KEY = lo

## Read from Athena

In [18]:
# read data from Athena
df = at.get_data_from_athena(region=region,  query_string=query_string, 
                                      database=database[0],
                                      workgroup=workgroup)

Checking query status
Current status: QUEUED
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUN

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## Persist as a parquet

In [19]:
df

,UNIT_NUM,PARTITION_KEY,VEHICLE_AGE_TOTAL_MONTHS,REGION,AREA,DISTRICT,LOCATION,UNIT_MAKE_CODE,UNIT_MODEL,UNIT_MODEL_YEAR,UNIT_CATEGORY,UNIT_SIZE,UNIT_BODY_TYPE,UNIT_FUEL_TYPE,ACT_VEH_INSERVDT,PRODUCT_LINE,UNIT_SOLD_DATE,flag_datatype,MILES
0,100001,201801,44,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,46108
1,100001,201802,45,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,46108
2,100001,201803,46,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,48116
3,100001,201804,47,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,48116
4,100001,201805,48,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,50347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4815294,ZSS63809,202101,49,0006-CORPORATE,0999-ADMINISTRATION AGGREGATE,6990-10-PENSKE FLEET MANAGEMENT,6990-10-PENSKE FLEET MANAGEMENT,HIN,268,2017,TRUCK,MEDIUM,1900 24,DIESEL,2017-01-01 00:00:00.000,CONTRACT MAINTENANCE - PEG,2021-02-17 00:00:00.000,model_build,17472
4815295,ZSS63809,202102,50,0006-CORPORATE,0999-ADMINISTRATION AGGREGATE,6990-10-PENSKE FLEET MANAGEMENT,6990-10-PENSKE FLEET MANAGEMENT,HIN,268,2017,TRUCK,MEDIUM,1900 24,DIESEL,2017-01-01 00:00:00.000,CONTRACT MAINTENANCE - PEG,2021-02-17 00:00:00.000,model_build,17491
4815296,ZSS63809,202103,51,0006-CORPORATE,0999-ADMINISTRATION AGGREGATE,6990-10-PENSKE FLEET MANAGEMENT,6990-10-PENSKE FLEET MANAGEMENT,HIN,268,2017,TRUCK,MEDIUM,1900 24,DIESEL,2017-01-01 00:00:00.000,CONTRACT MAINTENANCE - PEG,2021-02-17 00:00:00.000,model_build,17491
4815297,ZSS63809,202104,52,0006-CORPORATE,0999-ADMINISTRATION AGGREGATE,6990-10-PENSKE FLEET MANAGEMENT,6990-10-PENSKE FLEET MANAGEMENT,HIN,268,2017,TRUCK,MEDIUM,1900 24,DIESEL,2017-01-01 00:00:00.000,CONTRACT MAINTENANCE - PEG,2021-02-17 00:00:00.000,model_build,17491


In [20]:
path = ['s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Interim/location_master/']
s3_helper.persist_file_to_path(df=df,
                       path=path[0],
                       filetype='parquet')

ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column UNIT_NUM with type object')

## Read from Parquet

In [11]:
df2 = s3_helper.read_parquet_from_path(path=path[0])
df2

,location_master_key,corp_cd,corp_nm,corp_address1,corp_address2,corp_city,corp_state_code,corp_state_name,corp_zip,corp_county,...,rdist_maint_cost_class,parent_dist_sufx,location_original_key,dist_overhead_type,onsiteshop_flag,auto_svc_net_loc_flg,exists_fdstmst_flag,day_light_saving_flag,time_zone_cd,locn_temp_closed
0,9421,HPTL,PENSKE TRUCK LEASING CO.,2675 MORGANTOWN RD,,READING,PA,PENNSYLVANIA,19607,BERKS,...,5,10,9421,None,R,Y,Y,Y,E,N
1,10226,3000,PENSKE LOGISTICS INC,3401 ENTERPRISE PARKWAY,SUITE 200,BEACHWOOD,OH,OHIO,44122,CUYAHOGA,...,,None,10226,None,None,None,Y,Y,E,N
2,3410,0500,SEYMOUR,1565 E 4TH ST,,SEYMOUR,IN,INDIANA,472743245,JACKSON,...,None,,3410,None,None,None,Y,Y,D,N
3,9317,HPTL,PENSKE TRUCK LEASING CO.,2675 MORGANTOWN RD,,READING,PA,PENNSYLVANIA,19607,BERKS,...,6,11,1591,HIGH,None,,Y,Y,D,N
4,11909,HPTL,PENSKE TRUCK LEASING CO.,2675 MORGANTOWN RD,,READING,PA,PENNSYLVANIA,19607,BERKS,...,,,296,None,None,,Y,Y,E,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12661,7806,HPTL,PENSKE TRUCK LEASING CO.,RT 10 GREEN HILLS,,READING,PA,PENNSYLVANIA,19607,BERKS,...,7,10,2252,None,None,None,Y,Y,E,N
12662,9584,HPTL,PENSKE TRUCK LEASING CO.,2675 MORGANTOWN RD,,READING,PA,PENNSYLVANIA,19607,BERKS,...,6,10,9584,None,R,Y,Y,Y,E,N
12663,4402,3000,PENSKE LOGISTICS INC,3401 ENTERPRISE PARKWAY,SUITE 200,BEACHWOOD,OH,OHIO,44122,CUYAHOGA,...,,03,4402,None,None,None,Y,Y,D,N
12664,1862,HPTL,PENSKE TRUCK LEASING CO.,RT 10 GREEN HILLS,,READING,PA,None,19607,BERKS,...,8,11,1862,None,None,None,Y,Y,P,N
